In [1]:
# https://mne.tools/stable/auto_tutorials/forward/30_forward.html

import mne
from mne.datasets import sample
data_path = sample.data_path()

# the raw file containing the channel location + types
raw_fname = '../CAMCAN_MEG_100/CC110033/task/task_raw.fif'
# The paths to Freesurfer reconstructions
subjects_dir = data_path / 'subjects'
subject = 'fsaverage' # change it to use freesurfer's bem

In [2]:
trans = './test_scripts/CAMCAN_fsaverage_trans.fif'
info = mne.io.read_info(raw_fname)

    Read a total of 8 projection items:
        mag_ssp_upright.fif : PCA-mags-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v5 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v2 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v3 (1 x 306)  idle


In [3]:
info.keys()

dict_keys(['file_id', 'events', 'hpi_results', 'hpi_meas', 'subject_info', 'device_info', 'helium_info', 'hpi_subsystem', 'proc_history', 'meas_id', 'experimenter', 'description', 'proj_id', 'proj_name', 'meas_date', 'utc_offset', 'sfreq', 'highpass', 'lowpass', 'line_freq', 'gantry_angle', 'chs', 'dev_head_t', 'ctf_head_t', 'dev_ctf_t', 'dig', 'bads', 'ch_names', 'nchan', 'projs', 'comps', 'acq_pars', 'acq_stim', 'custom_ref_applied', 'xplotter_layout', 'kit_system_id'])

In [4]:
# Compute Source Space (surface)
# https://mne.tools/0.19/generated/mne.setup_source_space.html
# The spacing to use. Can be 'ico#' for a recursively subdivided icosahedron, 
# 'oct#' for a recursively subdivided octahedron, 
# 'all' for all points, or an integer to use appoximate distance-based spacing (in mm).
# src = mne.setup_source_space(subject, spacing='ico4', add_dist='patch',
#                              subjects_dir=subjects_dir)
# oct8 takes 2.5 min produces 65538 source
# ico7 = 'all'

# Compute Source Space (volume)
# https://mne.tools/0.19/generated/mne.setup_volume_source_space.html#mne.setup_volume_source_space
import os.path as op
bem = '/home/zhibinz2/mne_data/MNE-sample-data/subjects/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif'
src = mne.setup_volume_source_space(subject, pos=5.0, bem=bem, subjects_dir=subjects_dir, verbose=True)



BEM              : /home/zhibinz2/mne_data/MNE-sample-data/subjects/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif
grid                  : 5.0 mm
mindist               : 5.0 mm
MRI volume            : /home/zhibinz2/mne_data/MNE-sample-data/subjects/fsaverage/mri/T1.mgz

Reading /home/zhibinz2/mne_data/MNE-sample-data/subjects/fsaverage/mri/T1.mgz...

Loaded inner skull from /home/zhibinz2/mne_data/MNE-sample-data/subjects/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif (2562 nodes)
Surface CM = (  -0.5  -21.1    6.2) mm
Surface fits inside a sphere with radius   98.3 mm
Surface extent:
    x =  -75.2 ...   76.3 mm
    y = -113.3 ...   75.0 mm
    z =  -71.7 ...   88.2 mm
Grid extent:
    x =  -80.0 ...   80.0 mm
    y = -115.0 ...   75.0 mm
    z =  -75.0 ...   90.0 mm
43758 sources before omitting any.
29824 sources after omitting infeasible sources not within 0.0 - 98.3 mm.
Source spaces are in MRI coordinates.
Checking that the sources are inside the surface and at least    

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found   795/14601 points outside using solid angles
    Total 17459/29824 points inside the surface
Interior check completed in 6741.8 ms
    12365 source space points omitted because they are outside the inner skull surface.
    2830 source space points omitted because of the    5.0-mm distance limit.
14629 sources remaining after excluding the sources outside the surface and less than    5.0 mm inside.
Adjusting the neighborhood info.
Source space : MRI voxel -> MRI (surface RAS)
     0.005000  0.000000  0.000000     -80.00 mm
     0.000000  0.005000  0.000000    -115.00 mm
     0.000000  0.000000  0.005000     -75.00 mm
     0.000000  0.000000  0.000000       1.00
MRI volume : MRI voxel -> MRI (surface RAS)
    -0.001000  0.000000  0.000000     128.00 mm
     0.000000  0.000000  0.001000    -128.00 mm
     0.000000 -0.001000  0.000000     128.00 mm
     0.000000  0.000000  0.000000       1.00
MRI volume : MRI (surface RAS) -> RAS (non-zero origin)
     1.000000  0.000000  0.0000

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.7s finished


    12337105/16777216 nonzero values for the whole brain
[done]


In [13]:
# compute forward solution
# conductivity = (0.3,)  # for single layer
conductivity = (0.3, 0.006, 0.3)  # for three layers
model = mne.make_bem_model(subject='fsaverage', ico=4,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)
# compute all 3 layers (1m)

Creating the BEM geometry...
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
outer skin  CM is  -0.21 -19.38  -0.23 mm
outer skull CM is  -0.19 -19.34  -0.49 mm
inner skull CM is  -0.53 -21.10   6.21 mm
Checking that surface outer skull is inside surface outer skin  ...
Checking that surface inner skull is inside surface outer skull ...
Checking distance between outer skin  and outer skull surfaces...
Minimum distance between the outer skin  and outer skull surfaces is approximately    1.6 mm
Checking distance between outer skull and inner skull surfaces...
Minimum distance between the outer skull and inner skull surfaces is approximately    5.4 mm
Surfaces passed the basic topology checks.
Complete.

Three-layer model surfaces loaded.
Computing the linear collocation solution...
    Matrix coefficients...
     

In [5]:
fwd = mne.make_forward_solution(raw_fname, trans=trans, src=src, bem=bem,
                                meg=True, eeg=False, mindist=5.0, n_jobs=2,
                                verbose=True)
print(fwd)
#  1m17s
# compute 3 layers (36 s)

Source space          : <SourceSpaces: [<volume, shape=(33, 39, 34), n_used=14629>] MRI (surface RAS) coords, subject 'fsaverage', ~74.8 MB>


MRI -> head transform : ./test_scripts/CAMCAN_fsaverage_trans.fif
Measurement data      : task_raw.fif
Conductor model   : /home/zhibinz2/mne_data/MNE-sample-data/subjects/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Read 1 source spaces a total of 14629 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999927 -0.009207 -0.007775      -4.28 mm
     0.009519  0.999110  0.041099      24.90 mm
     0.007390 -0.041170  0.999125      31.33 mm
     0.000000  0.000000  0.000000       1.00

Read 306 MEG channels from info
105 coil definitions read
Coordinate transformation: MEG device -> head
     0.998429 -0.050574  0.024122       2.53 mm
     0.052468  0.994938 -0.085709     -15.30 mm
    -0.019665  0.086840  0.996028      39.95 mm
     0.000000  0.000000  0.000000       1.00
MEG coil definitions created in head coordinates.
Source spaces are now in head coordina

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found     0/10985 points outside using solid angles
    Total 14629/14629 points inside the surface
Interior check completed in 4584.5 ms

Checking surface interior status for 306 points...
    Found   0/306 points inside  an interior sphere of radius   83.4 mm
    Found  18/306 points outside an exterior sphere of radius  133.7 mm
    Found 288/288 points outside using surface Qhull
    Found   0/  0 points outside using solid angles
    Total 0/306 points inside the surface
Interior check completed in 37.3 ms

Composing the field computation matrix...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0

Computing MEG at 14629 source locations (free orientations)...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.0s finished



Finished.
<Forward | MEG channels: 306 | EEG channels: 0 | Source space: Volume with 14629 grid points | Source orientation: Free>


In [6]:
print(f'Before: {src}')
print(f'After:  {fwd["src"]}')

Before: <SourceSpaces: [<volume, shape=(33, 39, 34), n_used=14629>] MRI (surface RAS) coords, subject 'fsaverage', ~74.8 MB>
After:  <SourceSpaces: [<volume, shape=(33, 39, 34), n_used=14629>] head coords, subject 'fsaverage', ~216.0 MB>


In [7]:
leadfield = fwd['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

Leadfield size : 306 sensors x 43887 dipoles


In [8]:
fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True)
leadfield = fwd_fixed['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

ValueError: Forward operator was generated with sources from a volume source space. Conversion to fixed orientation is not possible. Consider using a discrete source space if you have meaningful normal orientations.

In [12]:
from hdf5storage import loadmat, savemat 
outdict=dict()
outdict['leadfield']=leadfield
outdict['source_nn']=fwd_fixed['source_nn']
outdict['source_rr']=fwd_fixed['source_rr']
savemat('leadfield_nn_rr',outdict)

In [10]:
mne.write_forward_solution('volume_fwd.fif', fwd, overwrite=True, verbose=None)

Overwriting existing file.
    Write a source space...
    [done]
    1 source spaces written


In [17]:
mne.write_forward_solution('simple_fsaverage_fixed_fwd.fif', fwd_fixed, overwrite=False, verbose=None)

    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written


C:\Users\zhouz\AppData\Local\Temp\ipykernel_24484\3691629218.py:1: RuntimeWarning: This forward solution is based on a forward solution with free orientation. The original forward solution is stored on disk in X/Y/Z RAS coordinates. Any transformation (surface orientation or fixed orientation) will be reverted. To reapply any transformation to the forward operator please apply convert_forward_solution after reading the forward solution with read_forward_solution.
  mne.write_forward_solution('simple_fsaverage_fixed_fwd.fif', fwd_fixed, overwrite=False, verbose=None)


In [19]:
# forward = mne.read_forward_solution('simple_fsaverage_fixed_fwd.fif',verbose=None)

Reading forward solution from /home/zhibinz2/Documents/GitHub/MEG_EEG_Source_Localization/simple_fsaverage_fwd.fif...
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read MEG forward solution (516 sources, 306 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame
